# Video games recommender system
based on steam sales datasets : 
- https://cseweb.ucsd.edu/~jmcauley/datasets.html#steam_data) gathered by [Insert teacher names]
- https://www.kaggle.com/nikdavis/steam-store-games

In this notebook we'll try to build a video games recommender system based on a steam sales dataset. In order to do so we'll use different techniques to recommend as precisely as possible a game to a user.

_______

## Imports

In [1]:
#Data manipulation
import pandas as pd
import numpy as np
import string
import re

#File manipulation
import os
import gzip
import ast

#DataViz
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

#Helper functions
%load_ext autoreload
%autoreload 2
import helper

## Dataframe Construction

Lets begin with the Australian dataset which contains user activity for steam user in Australia. 

In [154]:
data = list(helper.parseDataFromFile('data/australian_users_items.json'))

This dataset contains the information of **88,310** different steam users.

In [155]:
full_user = pd.concat([helper.user_dataframe(elt) for elt in data],axis = 0).reset_index(drop = True)

In [4]:
#full_user.head()

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0


In [156]:
full_user[['item_id','playtime_forever','playtime_2weeks']] = full_user[['item_id','playtime_forever','playtime_2weeks']].astype('int32')


Now, it could be usefull to add features to our dataset. We might find some in the steam dataset found on kaggle and a similar dataset found on https://cseweb.ucsd.edu/~jmcauley/datasets.html#steam_data, which regroups valuable information relative to games such as the price.

## Adding columns

In [157]:
games_data = list(helper.parseDataFromFile('data/steam_games.json'))
games = pd.DataFrame(games_data)
steam_df = pd.read_csv('data/steam.csv')

In [8]:
#games.head()

,app_name,developer,discount_price,early_access,genres,id,metascore,price,publisher,release_date,reviews_url,sentiment,specs,tags,title,url
0,Lost Summoner Kitty,Kotoshiro,4.49,False,"[Action, Casual, Indie, Simulation, Strategy]",761140,NaN,4.99,Kotoshiro,2018-01-04,http://steamcommunity.com/app/761140/reviews/?...,NaN,[Single-player],"[Strategy, Action, Indie, Casual, Simulation]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...
1,Ironbound,Secret Level SRL,NaN,False,"[Free to Play, Indie, RPG, Strategy]",643980,NaN,Free To Play,"Making Fun, Inc.",2018-01-04,http://steamcommunity.com/app/643980/reviews/?...,Mostly Positive,"[Single-player, Multi-player, Online Multi-Pla...","[Free to Play, Strategy, Indie, RPG, Card Game...",Ironbound,http://store.steampowered.com/app/643980/Ironb...
2,Real Pool 3D - Poolians,Poolians.com,NaN,False,"[Casual, Free to Play, Indie, Simulation, Sports]",670290,NaN,Free to Play,Poolians.com,2017-07-24,http://steamcommunity.com/app/670290/reviews/?...,Mostly Positive,"[Single-player, Multi-player, Online Multi-Pla...","[Free to Play, Simulation, Sports, Casual, Ind...",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...
3,弹炸人2222,彼岸领域,0.83,False,"[Action, Adventure, Casual]",767400,NaN,0.99,彼岸领域,2017-12-07,http://steamcommunity.com/app/767400/reviews/?...,NaN,[Single-player],"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/
4,Log Challenge,NaN,1.79,False,NaN,773570,NaN,2.99,NaN,NaN,http://steamcommunity.com/app/773570/reviews/?...,NaN,"[Single-player, Full controller support, HTC V...","[Action, Indie, Casual, Sports]",NaN,http://store.steampowered.com/app/773570/Log_C...


In [9]:
#steam_df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


Merging this two dataframe before adding them to the **full_user** dataframe could be useful.

In [158]:
steam_df_ = steam_df.drop(['appid','english','developer','publisher','platforms','required_age',
               'achievements','positive_ratings','negative_ratings','average_playtime',
              'median_playtime','owners'], axis = 1)

In [161]:
steam_df_.loc[:,'categories'] = steam_df_['categories'].apply(lambda x : x.split(sep = ';'))
steam_df_.loc[:,'genres'] = steam_df_['genres'].apply(lambda x : x.split(sep = ';'))
steam_df_.loc[:,'steamspy_tags'] = steam_df_['steamspy_tags'].apply(lambda x : x.split(sep = ';'))

In [162]:
games_ = games.drop(['id','developer','discount_price','early_access','metascore',
                     'publisher','reviews_url','sentiment','title','url'],axis = 1)

In [163]:
games_ = games_[['app_name','release_date','specs','genres','tags','price']]

In [164]:
games_.columns = steam_df_.columns

In [165]:
games_ = games_.dropna(axis = 0, subset = ['name'])

Many nan values in games are data present in **steam_df_**, so to make the majority of those nan disappear we just need to fill **games_** with **steam_df_**'s values.

In [166]:
games_ = games_.fillna(steam_df_)

In [167]:
final_games = pd.concat([games_,steam_df_],axis = 0).reset_index(drop = True)
final_games.shape

(59208, 6)

There might be a few duplicate values in this new dataframe.
Before using the **drop_duplicates** function on games' names, we might want to apply the **text_cleaning** function so that we are sure to eliminate every duplicates.

In [168]:
final_games['name'] = final_games['name'].astype(str)
helper.text_cleaning(final_games,'name')

In [169]:
final_games = final_games.drop_duplicates(subset = 'name')
final_games.shape

(43167, 6)

Now that the games' names have been cleaned in the **final_games** dataset, we need to do the same in **full_user**.

In [170]:
helper.text_cleaning(full_user,'item_name')

In [171]:
user_games = full_user.merge(final_games,how = 'left',left_on = 'item_name',
                             right_on = 'name').drop(['name'],axis = 1)

In [172]:
user_games.isna().sum()/len(user_games),len(user_games),len(full_user)

(user_id             0.000000
 item_id             0.000000
 item_name           0.000000
 playtime_forever    0.000000
 playtime_2weeks     0.000000
 release_date        0.209084
 categories          0.203795
 genres              0.211113
 steamspy_tags       0.202893
 price               0.211020
 dtype: float64, 5153209, 5153209)

Seems like we're gonna have to drop ±20% of our data in order to no longer have nan values.

In [173]:
final_df = user_games.dropna(axis = 0)

After dropping the nan values with still have more than 4 million rows which is largely enough.

In [ ]:
final_df.shape

Time to download the resulting dataframe in an easy to load file.

In [2]:
#final_df.to_pickle('data/final_df.pckl')

In [98]:
final_df = pd.read_pickle('data/final_df.pckl')

In [99]:
final_df.head()

,user_id,item_id,item_name,playtime_forever,playtime_2weeks,release_date,categories,genres,steamspy_tags,price
0,76561197970982479,10,counterstrike,6,0,2000-11-01,"[Multi-player, Valve Anti-Cheat enabled]",[Action],"[Action, FPS, Multiplayer, Shooter, Classic, T...",9.99
1,76561197970982479,20,teamfortressclassic,0,0,1999-04-01,"[Multi-player, Valve Anti-Cheat enabled]",[Action],"[Action, FPS, Multiplayer, Classic, Shooter, C...",4.99
2,76561197970982479,30,dayofdefeat,7,0,2003-05-01,"[Multi-player, Valve Anti-Cheat enabled]",[Action],"[FPS, World War II, Multiplayer, Action, Shoot...",4.99
3,76561197970982479,40,deathmatchclassic,0,0,2001-06-01,"[Multi-player, Valve Anti-Cheat enabled]",[Action],"[Action, FPS, Multiplayer, Classic, Shooter, F...",4.99
4,76561197970982479,50,halflifeopposingforce,0,0,1999-11-01,"[Single-player, Multi-player, Valve Anti-Cheat...",[Action],"[FPS, Action, Sci-fi, Singleplayer, Classic, S...",4.99


## Feature engineering 

### Price

**price** should be a float column.

In [100]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4020731 entries, 0 to 5153208
Data columns (total 10 columns):
user_id             object
item_id             int32
item_name           object
playtime_forever    int32
playtime_2weeks     int32
release_date        object
categories          object
genres              object
steamspy_tags       object
price               object
dtypes: int32(3), object(7)
memory usage: 291.4+ MB


Lot's of free games are not priced as **0** but as **'Free to play'** (for example), we need to change that.

In [101]:
final_df.loc[:,'price'] = final_df['price'].replace('.*Free.*|.*Demo.*|.*Install.*',0,regex = True)

In [102]:
final_df.loc[:,'price'] = final_df['price'].replace('^Starting at [$]{1}','',regex = True)

After looking for the **'third party'** priced game on the steam store we can see that it's also a free game.

In [103]:
final_df[final_df['price'] == 'Third-party']['item_name'].unique()

array(['peggleextreme'], dtype=object)

In [104]:
final_df.loc[:,'price'] = final_df['price'].replace('Third-party',0,regex = True)

Now we can convert the column in a float column.

In [105]:
final_df.loc[:,'price'] = final_df.loc[:,'price'].astype('float32')

### Release Date

Some games' release date isn't set in a good format. We are gonna have to find those games and go on steam to find there real realse date.

In [106]:
replace_dict = {
    'Coming Soon!' : '2018-08-28',
    'Not yet available' : '2018-12-03',
    'TBD' : '2016-01-01',
    'Coming soon..' : '2017-03-29'
}

In [107]:
final_df.loc[:,'release_date'] = final_df['release_date'].replace(replace_dict)

In [108]:
final_df.loc[:,'release_date'] = pd.to_datetime(final_df['release_date'])

### Categories

In [109]:
df_copy = final_df.copy()
df_copy.head()

,user_id,item_id,item_name,playtime_forever,playtime_2weeks,release_date,categories,genres,steamspy_tags,price
0,76561197970982479,10,counterstrike,6,0,2000-11-01,"[Multi-player, Valve Anti-Cheat enabled]",[Action],"[Action, FPS, Multiplayer, Shooter, Classic, T...",9.99
1,76561197970982479,20,teamfortressclassic,0,0,1999-04-01,"[Multi-player, Valve Anti-Cheat enabled]",[Action],"[Action, FPS, Multiplayer, Classic, Shooter, C...",4.99
2,76561197970982479,30,dayofdefeat,7,0,2003-05-01,"[Multi-player, Valve Anti-Cheat enabled]",[Action],"[FPS, World War II, Multiplayer, Action, Shoot...",4.99
3,76561197970982479,40,deathmatchclassic,0,0,2001-06-01,"[Multi-player, Valve Anti-Cheat enabled]",[Action],"[Action, FPS, Multiplayer, Classic, Shooter, F...",4.99
4,76561197970982479,50,halflifeopposingforce,0,0,1999-11-01,"[Single-player, Multi-player, Valve Anti-Cheat...",[Action],"[FPS, Action, Sci-fi, Singleplayer, Classic, S...",4.99


In [110]:
categories = list(set([a for b in df_copy['categories'].tolist() for a in b]))

In [111]:
del_list = [elt for elt in categories if len(elt) == 1]

In [143]:
def list_deletion(x):
    """
    Function to remove unecessary words from the column categories' content.
    """
    for elt in x:
        if elt in del_list:
            x.remove(elt)
    return x

In [145]:
df_copy['categories'].apply(lambda x : list_deletion(x))

AttributeError: 'str' object has no attribute 'remove'

In [148]:
df = pd.read_csv('data/steam.csv')

In [151]:
df['categories'].apply(lambda x : x.split(sep = ';'))

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99
5,60,Ricochet,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Valve Anti-Ch...,Action,Action;FPS;Multiplayer,0,2758,684,175,10,5000000-10000000,3.99
6,70,Half-Life,1998-11-08,1,Valve,Valve,windows;mac;linux,0,Single-player;Multi-player;Online Multi-Player...,Action,FPS;Classic;Action,0,27755,1100,1300,83,5000000-10000000,7.19
7,80,Counter-Strike: Condition Zero,2004-03-01,1,Valve,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,Action;FPS;Multiplayer,0,12120,1439,427,43,10000000-20000000,7.19
8,130,Half-Life: Blue Shift,2001-06-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player,Action,FPS;Action;Sci-fi,0,3822,420,361,205,5000000-10000000,3.99
9,220,Half-Life 2,2004-11-16,1,Valve,Valve,windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading...,Action,FPS;Action;Sci-fi,33,67902,2419,691,402,10000000-20000000,7.19
